In [1]:
# -*- coding: utf-8 -*-

In [2]:
import os
import pandas as pd 
from glob import glob

In [3]:
import sys
import time, re, pickle, itertools
import urllib3, json
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import re

In [4]:
import nltk
from __future__ import print_function, unicode_literals
from gensim.models.word2vec import Word2Vec
from sklearn.preprocessing import normalize as sknorm
import numpy as np
from operator import itemgetter
import html

In [5]:
from ckonlpy.tag import Twitter as ctwitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Twitter, Mecab,Kkma
from konlpy.utils import pprint

In [6]:
import chat_bot as cb
import Database_Handler as dh

In [7]:
ct = ctwitter()
ot = Twitter()
mecab = Mecab()
xxxx = ct._dictionary._pos2words
print (list(map(lambda x: len(xxxx[x]), xxxx)))

score_weights = {
    'num_nouns': -0.1,
    'num_words': -0.2,
    'no_noun': -1
}

def my_score(candidate):
    num_nouns = len([w for w,t in candidate if t == 'Noun'])
    num_words = len(candidate)
    no_noun = 1 if num_nouns == 0 else 0
    
    score = (num_nouns * score_weights['num_nouns'] 
             + num_words * score_weights['num_words']
             + no_noun * score_weights['no_noun'])
    return score

ct.set_selector(score_weights, my_score)

[512, 1464079, 5532]
[('이', 'Noun'), ('것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]
best: [('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]


## 언론사 리스트

> stopwords를 위해서

In [8]:
pressList = pd.read_excel('./data/presslist.xlsx')

with(open('./data/newspress.txt','w',encoding='utf-8')) as f:
    f.write('\n'.join(set(pressList['언론사'].tolist())))
def Stopwords(file):
    stopwords = open(file,'r',  encoding='utf-8').readlines()
    stopwords = list(map(lambda x:x.strip(), stopwords))
    return stopwords
stopwordList = Stopwords('./data/newspress.txt')

# 자연어 처리

## TextRank 

In [9]:
swords =['이제','너무','중인' ,'만큼', '마찬가지', '꼬집었', '기자','습니다','어요','이렇게', "아", "휴", 
         "아이구", "아이쿠", "아이고",'라고','지난','당시','오전','오후',"어", "나", "우리", "저희", "따라", "의해",'데일리',
         '이날','갑자기','같다','는데','다른','에서','하는','하게','이후','통해','이날','정말','다시','스스로','같은','지난주',
         '이번주','다음주','그런','이런','첫째','둘째','셋째','넷째','다섯째','여섯째','일곱째','여덟째','아홉째','열째','기성',
             '이었','라는','저번','이번','면서','이렇게','저렇게','그렇게','정근','판청','아르테',
            '앵커','아나운서','좌윤','주고', 'B씨','A씨','경우','서로','대로','지사','인근','스완','위해','대한',
            '통한','기성','영제','간다','대해','수준','젊은','스탠','이씨','헤아','하라','패스트','좌안','이토','2TV',
            '성은','산다','나요','선우','누나','억원','태수','그냥','사이','어마','위엄','남자','로서','우리','누리','관련',
            '윤종','다친','승헌','골망','있는','까지','반종빈','녹이','현재','세이','카스티','스티','물론','라덴','상대로',
            '여러','아미','아사다','이닝','유기','신서','티브이데일리','공동','윤이','치원','타고','박씨','루수','유력','타고',
            '하고','처음','김씨','연재','사람','다이어','이다','인근','니다','아직','이씨','탈린','윤계','이날',
            '이것','저것','그것','신잡','장영','고씨','박씨','사실','경정','손새','장의','미도','말했',
            '빈살','실제','분들','건너','최근','조씨','장씨','올해','내년','작년','지난해','여서','A군','B군',
            '고든','램지','진혁','심지어','D씨','최근','니다','협회','그리즈','미오','그것','메이','이재','윈터',
            '임오','레이먼','사는','잡고','이야','치치','있다','1호','2호','3호','4호','5호','6호','7호','8호','9호',
            '다양한','부분','심씨','트루시','트루','심씨','부분','전체','주씨','펀치','원투','A사','1차','2차','3차','4차',
            '5차','6차','7차','8차','9차','10차','계속','MK','매경닷컴','MK스포츠','티노','처음','모현','들이','있는',
            '더욱','차장','부장','시점','000','달라','모든','구름','맑음','방향','같아','믹스','나인','윤균','모두',
            '라며','헤아','제트','라카','소속','1분','신잡','울스','일단','번째','어서','이지','원장','교수','선수',
            '안젤리나','브래드','드려','조원','둥지','빅리그','어떻','대답했','루프','트리플','빅토르','지금','때문','해성',
             '빠졌','그리팅','5일','점프','그룹','시즌','크게','없는']
swords2 = ['중인' ,'만큼', '마찬가지', '꼬집었', "기자","아", "휴", "아이구",
                      "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "가"] + pressList['언론사'].tolist() + ["저", "것",
    "동시에", "몇", "고려하면", "관련이", "놀라다", "무엇", "어느쪽", "오", "정도의", "더구나", "아무도", "줄은모른다", "참", "아니",
         "휘익", "향하다", "응당", "알겠는가", "인젠", "그래서", "자신", "해서는", "둘", "이었다", "임에", "하도록시키다", "누구", "이때",
         "삼", "제외하고", "쿵", "하면", "좀", "그렇지않으면", "아니었다면", "이라면", "팍", "일", "통하여", "무엇때문에", 
         "보아", "하게하다", "하는", "이르다", "타다", "까지도", "오직", "도달하다", "잠깐", "외에", "심지어", "하려고하다", "게다가",
         "후", "알", "비하면", "헉헉", "근거로", "월", "따라서", "않는다면", "일지라도", "함께", "이유는", "흥", "혼자", "관하여", 
         "붕붕", "하다", "진짜로", "의해", "바와같이", "대하면", "퍽", "보다더", "그렇게", "끼익", "댕그", "시초에", "당장", 
         "하는것만", "누가", "만이", "만일", "이지만", "하마터면", "꽈당", "만은", "우선", "없다", "휴", "하도록하다", "그런데",
         "비로소", "하게될것이다", "만큼 어찌됏든", "오히려", "을", "더라도", "안", "왜냐하면", "습니다", "줄은", "그리하여", "하",
         "어떻게", "대로", "기대여", "끙끙", "예를", "와르르", "이리하여", "이", "조차", "하고", "이젠", "뒤이어", "할줄알다", "반대로",
         "시각", "펄렁", "잇따라", "공동으로", "비록", "가까스로", "여덟", "비슷하다", "이상", "차라리", "이어서", "모두", "툭", "조차도",
         "헉", "부터", "혹시", "않고", "우리", "삐걱", "여보시오", "허", "해요", "견지에서", "하기는한데", "토하다", "않으면", "이봐", "관계가", 
         "한다면", "시작하여", "연이서", "이외에도", "그", "운운", "에게", "그럼에도", "예", "만약에", "했어요", "결과에", "제", "오자마자",
         "것들", "약간", "것과", "일때", "셋", "각종", "아이구", "같은", "향해서", "일것이다", "해야한다", "아이야", "로", "편이", "등등", 
         "해도좋다", "하기에", "김에", "몰랏다", "같이", "하도다", "즉시", "갖고말하자면", "우에", "어느", "허허", "하자마자", "에서", "그래도",
         "하여야", "된이상", "까악", "한켠으로는", "많은", "그중에서", "사", "낼", "뿐만", "저쪽", "어쩔수", "어떤것들", "물론", "결론을", "이만큼",
         "이렇게되면", "소인", "바꾸어말하면", "들", "이렇구나", "하물며", "얼마간", "얼마든지", "한항목", "하는것도", "졸졸", "한마디", "말할것도", 
         "만약", "남들", "총적으로", "허걱", "그리고", "따지지", "구체적으로", "못하다    하기보다는", "언제", "따르는", "구토하다", "앞에서",
         "대해서", "아", "앞의것", "비걱거리다", "헐떡헐떡", "어찌하든지", "입장에서", "의", "마저", "바로", "하기만", "않기", "또한", "쓰여",
         "위해서", "의거하여", "인", "아니면", "를", "사람들", "할수있다", "일곱", "근거하여", "한적이있다", "함으로써", "낫다", "어떤것",
         "방면으로", "중의하나", "어", "무릎쓰고", "저것만큼", "서술한바와같이", "그런즉", "들자면", "하지", "아이고", "불문하고", "만", 
         "마저도", "얼마만큼", "예컨대", "이렇게말하자면", "연관되다", "않다면", "들면", "이쪽", "의지하여", "여섯", "그저", "아니다", 
         "그렇지만", "기준으로", "되어", "가", "무렵", "즉", "말하면", "어찌", "그럼", "그위에", "그런", "조금", "매번", "혹은",
         "이천구", "중에서", "따름이다", "하기", "가령", "잠시", "아무거나", "하기보다는", "주저하지", "당신", "봐라", "그렇지",
         "자기집", "할지라도", "요만한걸", "우르르", "못하다", "왜", "이렇게", "퉤", "관계없이", "그래", "대해", "쪽으로", 
         "저것", "자기", "아홉", "지만", "구", "하지마", "따위", "하지만", "나", "해도", "전자", "그만이다", "안된다", "까닭으로", 
         "되다", "오르다", "딱", "다음에", "너희들", "점에서", "아이쿠", "쾅쾅", "종합한것과같이", "할수있어", "그치지", "비교적",
         "륙", "되는", "개의치않고", "엉엉", "하든지", "때가", "영차", "바꿔", "더불어", "주룩주룩", "따라", "이용하여", "우리들",
         "여기", "더욱이는", "하더라도", "입각하여", "여러분", "마치", "하느니", "너", "어디", "제각기", "밖에", "봐", "위하여", 
         "팔", "요만큼", "가서", "아니라면", "지든지", "참나", "할만하다", "타인", "든간에", "하겠는가", "거바", "겨우", "다음", 
         "이러한", "이럴정도로", "각자", "어때", "지말고", "형식으로", "그러한즉", "아니나다를가", "할", "불구하고", "지경이다",
    "어떠한", "기점으로", "할때", "등", "다시", "시키다", "답다", "소생", "라", "로써", "각", "부류의", "알았어", "훨씬", "위에서",
    "뿐이다", "시간", "그러나", "하곤하였다", "일단", "막론하고", "좋아", "솨", "이곳", "뿐만아니라", "아울러", "옆사람", "다수", 
    "예하면", "령", "어떤", "어떻해", "할수록", "말하자면", "전후", "메쓰겁다", "에", "으로써", "이번", "하면된다", "이것", "딩동",
"양자", "달려", "본대로", "탕탕", "마음대로", "쉿", "미치다", "다시말하면", "동안", "그러니까", "과연", "뚝뚝", "거의", "이천팔",
"이로", "않도록", "또", "한하다", "아래윗", "수", "다소", "어느것", "까지", "남짓", "저기", "관한", "무슨", "그에", "년도", 
"삐걱거리다", "이러이러하다", "와", "넷", "쳇", "논하지", "습니까", "이천육", "기타", "오로지", "어느곳", "설령", "할지언정", "칠",
"다만", "반드시", "한데", "곧", "의해서", "얼마나", "아니라", "상대적으로", "너희", "있다", "인하여", "다섯", "생각이다", "몰라도", 
"정도에", "버금", "까닭에", "얼마큼", "전부", "로부터", "힘입어", "틈타", "해도된다", "나머지는", "흐흐", "그때", "하여금", "모", 
"이런", "바꾸어서", "비추어", "각각", "설사", "이래", "비길수", "하지마라", "응", "다른", "듯하다", "보는데서", "어쨋든", "대하여", 
"좍좍", "으로", "여차", "틀림없다", "과", "고로", "요컨대", "일반적으로", "줄", "하는바", "그들", "요만한", "윙윙", "콸콸", "어기여차",
"언젠가", "이와", "할망정", "이천칠", "네", "없고", "둥둥", "겸사겸사", "그러므로", "안다", "거니와", "년", "여부", "때문에", "된바에야",
"향하여", "때", "하하", "및", "오호", "하면서", "더군다나", "한", "이유만으로", "어이", "하나", "저희", "더욱더", "두번째로", "바꾸어말하자면",
"이와같다면", "이르기까지", "단지", "그러면", "야", "결국", "영","뒤따라", "즈음하여", "도착하다", "와아", "다음으로", "같다", "자",
"아하", "생각한대로", "외에도", "의해되다", "설마", "으로서", "보면", "할뿐", "첫번째로", "아야", "어째서", "하는것이", "하구나", "않다",
"힘이", "육", "그러니", "여전히", "어찌됏어", "어찌하여", "어느해", "앗", "게우다","보드득", "관해서는", "자마자", "매", "하고있었다", 
"어느때", "여", "실로", "해봐요", "얼마", "아이"]

In [ ]:

class SentenceTokenizer(object):
    def __init__(self):
        self.twitter = ot
        self.ctwitter = ct
        self.mecab = mecab
        
        #self.stopwords = []
        self.stopwords = swords +swords2 + stopwordList
    def text2sentences(self, text):
        sentences = re.sub('\. ','.\n', text).split('\n')
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.ctwitter.phrases(str(sentence))
                                                    if noun not in self.stopwords and len(noun) > 1]))
                nouns.append(' '.join([noun for noun in self.ctwitter.nouns(str(sentence))
                                                  if noun not in self.stopwords and len(noun) > 1]))
                nouns.append(' '.join([noun for noun in self.ctwitter.morphs(str(sentence))
                                                   if noun not in self.stopwords and len(noun) > 1]))
                #nouns.append(' '.join([noun for noun in self.mecab.nouns(str(sentence))
                #                                  if noun not in self.stopwords and len(noun) > 1]))
                #nouns.append(' '.join([noun for noun in self.mecab.phrases(str(sentence))
                #                                  if noun not in self.stopwords and len(noun) > 1]))
                #nouns.append(' '.join([noun for noun in self.mecab.morphs(str(sentence))
                #                                   if noun not in self.stopwords and len(noun) > 1]))
        nouns = list(set(nouns))
        return nouns
    
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer(ngram_range=(1,4),sublinear_tf=True, lowercase=False, min_df=0.0001)
        self.cnt_vec = CountVectorizer(ngram_range=(1,4), lowercase=False,min_df=0.0001,)
        self.graph_sentence = []
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        #tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    def build_words_graph(self, sentence):
        cnt_vec_mat = sknorm(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

class TextRankX(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        return summary
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        return keywords

# 분석

* 중복되서 나오는 것 제거
* 혼자 나왔는데 태그가 이상한거 제거

In [ ]:
extract_keywords_dict = dict()
site = 'Naver'
#site = 'daum'
if site == 'daum':
    binList = []
    collection = 'newsDaum'
    etri_outcome = '/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/data/ETRI_OUTCOME/daum2/'
elif site.lower() == 'naver':
    binList = []
    collection = 'newsNaver'
    etri_outcome = '/Users/hyunyoun/Documents/GitHub/Private_Project/personal_project/data/ETRI_OUTCOME/naver2/'

mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
slack = cb.Slacker(cb.slacktoken())
useCollection = dh.Use_Collection(useDb, collection)
keywordList = list()
for data in useCollection.find({'site':site}):
    #if len(data['mainText'].strip()) == 0:
        #print (data['link'])
        #break
    idIs = data['_id']._ObjectId__id
    tis2 = html.unescape(data['title'])+'. '+html.unescape(data['mainText'])
    #tis2 = '.\n'.join(tis.split('. '))
    #tis2 = data['mainText']
    if data['keywords'] !='NaN':
        keywordList += list(map(lambda x: ''.join(x.split(' · ')[-1].split(' ')), data['keywords']))
    else:
        pass
    print (data['link'])
    print (data['title'])
    print (data['keywords'])
    try:
        workrank = TextRankX(tis2)
    except:
        try:
            workrank = TextRankX(tis2)
        except:
            workrank = TextRankX(tis2)
            break
    extract_keywords_dict[idIs.hex()]= workrank.keywords()
    print (workrank.keywords())
    #print (lr)
    print ()
    
    '''
    if not os.path.isfile(etri_outcome+idIs.hex()+'.picked.txt'):
        try:
            etri = USE_ETRI_ANALYSIS('srl', tis2)
        except:
            print (etri[1])
            break
        else:
            pickle.dump(etri[0], open(etri_outcome+idIs.hex()+'.picked.txt','wb'))
            '''



http://sports.news.naver.com/sports/index.nhn?ctg=ranking_news&mod=read&ranking_type=popular_day&date=20171212&office_id=410&article_id=0000425760
[윈터미팅 1일차] 모리스의 눈물, 스탠튼의 뒤끝
['마이애미 말린스', '지안카를로 스탠튼']
['말린스', '전당', '양키스', '이적', '명예의 전당', '명예의', '스탠튼', '기자들', '명예', '명예 명예의 전당']

http://sports.news.naver.com/sports/index.nhn?ctg=ranking_news&mod=read&ranking_type=popular_day&date=20171212&office_id=477&article_id=0000102428
[SPO 이슈] '2% 확률' 니퍼트 재취업 가능성은?
NaN
['외국인', '니퍼트', '롯데', '두산', '외국인 선발투수', '영입', '선발투수', '계약', 'kt', '삼성']

http://sports.news.naver.com/sports/index.nhn?ctg=ranking_news&mod=read&ranking_type=popular_day&date=20171212&office_id=452&article_id=0000000730
[서호정의 킥오프] 믿을 건 황금세대뿐, 한국 여자 축구의 현실
NaN
['한국', '여자', '축구', '북한', '대표팀', '월드컵', '여자 축구', '감독', '선수들', '축구 여자']

http://sports.news.naver.com/sports/index.nhn?ctg=ranking_news&mod=read&ranking_type=popular_day&date=20171212&office_id=139&article_id=0002084547
"우리가 바카요코를 좀 도와주자" 캉테 호소
['은골로 캉테']
['바카 요코', '요코', '바카', '바카요

['카풀', '서비스', '택시', '퇴근', '규제', '카풀 서비스', '서울시', '차량', '카풀앱', '택시업계']

http://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=015&aid=0003863698&date=20171212&type=1&rankingSectionId=105&rankingSeq=10
[사이언스] "1770년 동북아 밤하늘을 9일이나 붉게 물들인 건 오로라였다"
['동북아', '동아시아', '비']
['자기폭풍', '오로라', '기록', '관측', '현상', '규모', '캐링턴', '당시', '문헌', '캐링턴 사건']

http://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=025&aid=0002780468&date=20171212&type=1&rankingSectionId=105&rankingSeq=8
무한진화 AI 생태계 “한국은 바닥서 쫓겨다니는 토끼”
['인공지능', 'ai', '생태계']
['인공지능', '인공', '인공 지능', '지능', '기술', '인공지능 기술', '한국', '투자', '소프트웨어', '지능 기술']

http://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=028&aid=0002390543&date=20171212&type=1&rankingSectionId=105&rankingSeq=2
 인터넷 통신사 바꿀 때 해지신청 안 하면…요금은? 
['번호이동', '이동통신', '인터넷']
['해지', '사업자', '신청', '요금', '초고속인터넷', '대부분', '텔레비전', '자동', '인터넷', '고속']

http://news.naver.com/main/ranking/read.nhn?mid=

['서울', '기온', '북극진동', '북극', '지구온난화', '기상청', '한파', '지역', '기단', '영하']

http://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=003&aid=0008338480&date=20171212&type=1&rankingSectionId=102&rankingSeq=30
일자리 통계도 '맞춤형'…베이비부머·사회적경제 新통계 개발
['일자리', '일자리위원회', '세종시']
['일자리', '통계', '경제', '대상', '고용', '방안', '공표', '세종', '정책', '정부']

http://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=005&aid=0001055043&date=20171212&type=1&rankingSectionId=102&rankingSeq=29
[집과 삶, 그리고 주거권] “집 있어 다행?… 라면 하나 사먹기도 힘들어”
['라면', '단독주택', '고정비']
['할아버지', '서울', '만원', '난방', '기초', '제대로', '이사', '기준', '단독주택', '월소득']

http://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=001&aid=0009744377&date=20171212&type=1&rankingSectionId=102&rankingSeq=28
경찰, '비트코인 사기극' 논란 고교생 신변보호
['비트코인', '가상화폐', '스마트워치']
['경찰', '신변보호', '비트코인', '스마트워치', '신변', 'A군', '경찰서', '요청', '신변보호 요청', '11일']

http://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=

['원내대표', '한국당', '의원', '홍준표', '당내', '원내', '정부', '국민의당', '정책', '계파']

http://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=437&aid=0000169876&date=20171212&type=2&rankingSectionId=100&rankingSeq=22
한국당 새 원내대표에 '친홍' 김성태…당 개혁 속도 전망
['김성태', '한국당', '친홍']
['의원', '원내대표', '홍준표', '홍준표 대표', '대표', '속도', '김성태', '한국당', '자유', '대표 홍준표']

http://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=001&aid=0009744743&date=20171212&type=1&rankingSectionId=100&rankingSeq=18
한중 정상회담 후 '공동언론발표' 아닌 '언론발표'
['한중정상회담', '사드', '정상회담']
['양국', '청와대', '형식', '언론발표', '대통령의', '중국', '내용', '방중', '사드', '한국과 중국']

http://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=437&aid=0000169779&date=20171212&type=2&rankingSectionId=100&rankingSeq=17
[단독] '박근혜 특활비 상납' 시작에 최경환 관여 정황
['최경환', '박근혜', '국가정보원']
['의원', '상납', '특활비', '대통령', '특활비 상납', '국정원', '정황', '박근혜', '최경환', '대통령 대통령']

http://news.naver.com/main/ranking/read.nhn?mid=etc&sid

In [ ]:
pickle.dump(extract_keywords_dict, open('./data/Keyword_outcome/ckonlpy-naver.pickled','wb'))

* 형태소 분석 : morp
* 어휘의미 분석 : wsd (동음이의어)
* 어휘의미 분석 : wsd_poly (다의어)
* 개체명 인식 : ner
* 의존구문 분석 : dependency
* 의미역 인식 : srl

# sentiment analysis